# setup and data

In [264]:
import numpy as np

import pandas as pd
from pandas import Series,DataFrame
from bokeh.layouts import gridplot, row, column

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure

from sklearn.datasets import load_boston

from bokeh.models import (BasicTicker, ColumnDataSource, Grid, LinearAxis,
                         DataRange1d, PanTool, Plot, WheelZoomTool)
import sklearn.model_selection

# For very simple visualizations (without too much interaction, output_notebook works too, which we enable here)
output_notebook()

Loading BokehJS ...

In [290]:
# Load dataset
df18 = pd.read_csv('../Data_csv/data_2018.csv')
for columnname in df18:
    print(columnname)
df18['stats_pc_intl_students'] = df18['stats_pc_intl_students'].map(lambda x: x.rstrip('%'))
df18 = df18
        
rank = df18['rank']
location = df18['location']
percentage_male = df18['percentage_male']
stats_number_students = df18['stats_number_students']
stats_student_staff_ratio = df18['stats_student_staff_ratio']

# Data Columns
# haal alle data uit de database waar we niet naar kijken
df18new = df18.drop('url', axis=1)
df18new = df18new.drop('subjects_offered', axis=1)
df18new = df18new.drop('name', axis=1)
df18new = df18new.drop('location', axis=1)
df18new = df18new.drop('male_students', axis=1)
df18new = df18new.drop('nid', axis=1)
df18new = df18new.drop('scores_industry_income_rank', axis=1)
df18new = df18new.drop('scores_international_outlook_rank', axis=1)
df18new = df18new.drop('scores_research_rank', axis=1)
df18new = df18new.drop('scores_teaching_rank', axis=1)
df18new = df18new.drop('scores_citations_rank', axis=1)
df18new = df18new.drop('Unnamed: 0', axis=1)
df18new = df18new.drop('rank', axis=1)
df18new = df18new.drop('rank_order', axis=1)
df18new = df18new.drop('stats_number_students', axis=1)

df18new = df18new.drop('percentage_male', axis=1)
df18new = df18new.drop('scores_overall', axis=1)

columnlist = [location, percentage_male, stats_number_students, stats_student_staff_ratio, rank]

Unnamed: 0
location
name
nid
rank
rank_order
scores_citations
scores_citations_rank
scores_industry_income
scores_industry_income_rank
scores_international_outlook
scores_international_outlook_rank
scores_overall
scores_research
scores_research_rank
scores_teaching
scores_teaching_rank
percentage_male
stats_number_students
stats_pc_intl_students
stats_student_staff_ratio
subjects_offered
url
male_students


In [289]:
figures = [figure() for _ in range(6)]
for fig, column in zip(figures, df18new):
    # compute line
    
    Y = df18new.stats_pc_intl_students
    X = np.vstack(df18new[column])
    X = np.column_stack((X, np.ones(X.shape[0])))

    a, b = np.linalg.lstsq(X, Y)[0]
    
    # Create a scatter-plot
    fig.scatter(df18new[column], df18new["stats_pc_intl_students"])
    fig.line(x, a * x + b, color='red')
    
    # set yrange to always from 0 to 1103
    fig.y_range=DataRange1d(start=0, end=100)
    fig.x_range=DataRange1d(start=0, end=100)
    
    ## Add some axis information
    fig.xaxis.axis_label = column
    fig.yaxis.axis_label = 'international students'

    
show(gridplot(figures, ncols=2, plot_width=400, plot_height=250, toolbar_location=None))

# linear regression

## univariate linear regression
niet interessant voor onze échte regression

In [267]:
Y = df18.rank_order
X = np.vstack(df18new.scores_citations_rank)
X = np.column_stack((X, np.ones(X.shape[0])))

a, b = np.linalg.lstsq(X, Y)[0]

f = figure(plot_width=400, plot_height=250)

# Create a scatter-plot
f.scatter(df18new["scores_citations_rank"], df18new["rank_order"])
    
# Create the line
x = df18new['scores_citations_rank']
f.line(x, a * x + b, color='red')

## Add some axis information
f.xaxis.axis_label = "scores_citations_rank"
f.yaxis.axis_label = "overall rank"

show(f)

AttributeError: 'DataFrame' object has no attribute 'scores_citations_rank'

## computing the error
nog steeds niet interessant

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rmsecolumn = []

# calculate all mse's
for column in df18:
    try:
        rmse = sqrt(mean_squared_error(np.array(df18[column]),df18['rank_order']))
#         print(mse)
        rmsecolumn.append((rmse,column))
    except ValueError:
        print(column + ' ..whoeps')
        

# find lowest
sorted(rmsecolumn)

# r = np.array(df18['rank_order'])
# y = np.array(df18['rank'])
# mse = mean_squared_error(r, y)
# print(mse)

# multivariate regression

In [291]:
# Import for Linear Regression
import sklearn
from sklearn.linear_model import LinearRegression

# Create a LinearRegression Object
lreg = LinearRegression()

In [292]:
# creeer de multi-variabele X
X_multi = df18new.drop('stats_pc_intl_students',axis=1)

for name in df18new:
    print(name)

# Te vergelijken variabele, in ons geval aantal internationale studenten
Y_target = df18new['stats_pc_intl_students']

# Implement Linear Regression
lreg.fit(X_multi,Y_target)

scores_citations
scores_industry_income
scores_international_outlook
scores_research
scores_teaching
stats_pc_intl_students
stats_student_staff_ratio


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [304]:
# What is our constant? (b in univariate regression)
print(' The estimated intercept coefficient is {0:.2f} (is dan w0 van het voorbeeld)'.format(lreg.intercept_))

print(' The number of coefficients used was {0:d}'.format(len(lreg.coef_)))

 The estimated intercept coefficient is -10.99 (is dan w0 van het voorbeeld)
 The number of coefficients used was 6


In [294]:
coeff_df = DataFrame(df18new.drop('stats_pc_intl_students',axis=1).columns)
coeff_df.columns = ['features']

coeff_df["Coefficient Estimate"] = pd.Series(lreg.coef_)

coeff_df.sort_values(by='Coefficient Estimate', ascending=False)

,features,Coefficient Estimate
2,scores_international_outlook,0.441974
4,scores_teaching,0.204268
1,scores_industry_income,0.003398
5,stats_student_staff_ratio,-0.004322
0,scores_citations,-0.058728
3,scores_research,-0.070827


## training and test set

In [310]:
import sklearn.model_selection
# Grab the output and set as X and Y test and train data sets!
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X_multi,df18new.stats_pc_intl_students)

# Print shapes of the training and testing data sets
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(827, 6) (276, 6) (827,) (276,)


## the prediction

In [311]:
# Create our regression object
lreg = LinearRegression()

# Once again do a linear regression, except only on the training sets this time
lreg.fit(X_train,Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [312]:
# Predictions on training and testing sets
pred_train = lreg.predict(X_train)
pred_test = lreg.predict(X_test)

In [313]:
from sklearn.metrics import mean_squared_error

print("Fit a model X_train, and calculate MSE with Y_train: {0:.2f}".format(sqrt(mean_squared_error(Y_train, pred_train))))
    
print("Fit a model X_train, and calculate MSE with X_test and Y_test: {0:.2f}".format(sqrt(mean_squared_error(Y_test, pred_test))))

Fit a model X_train, and calculate MSE with Y_train: 6.12
Fit a model X_train, and calculate MSE with X_test and Y_test: 7.09
